In [ ]:
import pygame
import numpy as np
import sys
import math

# --- OBJ 파일 파서 ---
def load_obj(file_path):
    vertices = []
    edges = set()

    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('v '):
                parts = line.strip().split()
                x, y, z = map(float, parts[1:4])
                vertices.append(np.array([x, y, z]))
            elif line.startswith('f '):
                parts = line.strip().split()
                indices = [int(p.split('/')[0]) - 1 for p in parts[1:]]
                for i in range(len(indices)):
                    a = indices[i]
                    b = indices[(i + 1) % len(indices)]
                    edge = tuple(sorted((a, b)))
                    edges.add(edge)

    return vertices, list(edges)

# --- 회전 행렬 ---
def rotate_x(angle):
    cos_theta = math.cos(angle)
    sin_theta = math.sin(angle)
    return np.array([
        [1, 0, 0],
        [0, cos_theta, -sin_theta],
        [0, sin_theta, cos_theta]
    ])

def rotate_y(angle):
    cos_theta = math.cos(angle)
    sin_theta = math.sin(angle)
    return np.array([
        [cos_theta, 0, sin_theta],
        [0, 1, 0],
        [-sin_theta, 0, cos_theta]
    ])

def rotate_z(angle):
    cos_theta = math.cos(angle)
    sin_theta = math.sin(angle)
    return np.array([
        [cos_theta, -sin_theta, 0],
        [sin_theta, cos_theta, 0],
        [0, 0, 1]
    ])

# --- 카메라 뷰 행렬 ---
def look_at(camera_pos, camera_target, camera_up):
    forward = camera_target - camera_pos
    forward /= np.linalg.norm(forward)

    right = np.cross(camera_up, forward)
    right /= np.linalg.norm(right)

    up = np.cross(forward, right)
    mat = np.array([right, up, forward])
    return mat

# --- 3D → 2D 투영 ---
def project(point3d, distance):
    factor = 500 / (distance + point3d[2])
    x = point3d[0] * factor + width // 2
    y = -point3d[1] * factor + height // 2
    return (int(x), int(y))

# --- 초기화 ---
pygame.init()
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("OBJ Viewer")
clock = pygame.time.Clock()

# --- 모델 로드 ---
vertices, edges = load_obj(r"C:\Users\user\OneDrive\Desktop\Brilliant Snaget\tinker.obj")

# --- 카메라 세팅 ---
camera_distance = 100.0
camera_pos = np.array([0.0, 0.0, -camera_distance])
camera_target = np.array([0.0, 0.0, 0.0])
camera_up = np.array([0.0, 1.0, 0.0])

angle_x = angle_y = angle_z = 0.0

# --- 메인 루프 ---
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        elif event.type == pygame.MOUSEWHEEL:
            camera_distance -= event.y * 5
            camera_distance = max(10, min(300, camera_distance))
            camera_pos = np.array([0.0, 0.0, -camera_distance])

    # 마우스 위치 기반 회전
    mx, my = pygame.mouse.get_pos()
    angle_x = (my - height // 2) * 0.005
    angle_y = (mx - width // 2) * 0.005

    # 회전 및 뷰 변환
    rotation = rotate_x(angle_x) @ rotate_y(angle_y) @ rotate_z(angle_z)
    view_matrix = look_at(camera_pos, camera_target, camera_up)

    transformed_vertices = []
    for v in vertices:
        rotated = rotation @ v
        camera_space = view_matrix @ (rotated - camera_pos)
        projected = project(camera_space, camera_distance)
        transformed_vertices.append(projected)

    # 렌더링
    screen.fill((0, 0, 0))
    for edge in edges:
        try:
            pygame.draw.line(screen, (255, 255, 255),
                             transformed_vertices[edge[0]],
                             transformed_vertices[edge[1]], 1)
        except IndexError:
            continue

    pygame.display.flip()
    clock.tick(60)


ModuleNotFoundError: No module named 'pygame'